###                                      **Sentiment Analysis**

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
df=pd.read_csv('/content/Reviews.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,61946.000000,61946.000000,61946.000000,61946.000000,6.194600e+04
mean,30973.500000,1.632955,2.112081,4.152375,1.295148e+09
std,17882.414225,5.442560,6.123128,1.328556,4.744299e+07
min,1.000000,0.000000,0.000000,1.000000,9.617184e+08
25%,15487.250000,0.000000,0.000000,4.000000,1.269907e+09
50%,30973.500000,0.000000,1.000000,5.000000,1.309046e+09
75%,46459.750000,2.000000,2.000000,5.000000,1.331078e+09
max,61946.000000,398.000000,401.000000,5.000000,1.351210e+09


In [30]:
# @title Default title text
df[['Text','onverted_text']].head(20)
df[['Text','onverted_text']].to_csv('Reviews_Changed.csv')

In [10]:
import plotly.express as px

score_counts = df.groupby('Score').size().reset_index(name='Count')

fig = px.bar(score_counts, x='Score', y='Count',
             title='Count of Reviews under each score',
             labels={'Score': 'Score', 'Count': 'Number of Reviews'})
fig.show()




## Data Preprocessing

In [9]:
df.dropna(inplace=True)

Case Conversion

In [14]:
def convert_lower(df,text):
  df['onverted_text']=df[text].str.lower()
  return df

Remove Numerical Values

In [15]:
def remove_numerics(text):
    num = re.compile(r'[0-9]')
    return num.sub(r'', text)

Remove URLs

In [16]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

Remove Stopwords

In [17]:
def remove_stopwords(text):
  #print("I am here :",text)
  st=" ".join([i for i in str(text).split() if i not in STOPWORDS])
  return st

Remove Punctuations

In [18]:
def remove_punctuation(text):
  st=''.join([i for i in text if i not in string.punctuation])
  return st

Remove emojis

In [19]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

Remove emoticons

In [20]:
def remove_emoticons(text):
    emoticon_pattern = re.compile(r'[<>]?[:;=8][\-o\*\']?[\)\]\(\[dDpP/\:\}\{@\|\\]|[<>]?[\)\]\(\[dDpP/\:\}\{@\|\\][\-o\*\']?[:;=8]')
    return emoticon_pattern.sub(r'', text)


Remove html tags

In [21]:
from bs4 import BeautifulSoup

def remove_html(text):
    return BeautifulSoup(text, "lxml").text

Chat word Conversion

In [ ]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [ ]:
chat_word_list=[]
chat_words_map_dict = {}
for i in chat_words_str.split('\n'):
  if i!='':
    cw = i.split('=')[0]
    cw_expanded = i.split('=')[1]
    chat_word_list.append(cw)
    chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_word_list)
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

Perform Stemming

In [22]:
def stmng(para):
  tkn_para=para
  # convert Paragraph to list of sentences
  sentences = nltk.sent_tokenize(para)
  stemmer = nltk.PorterStemmer()
  # Convert Sentences to words

# Stemming
  for i in range(len(sentences)):
      words = nltk.word_tokenize(sentences[i])
      words = [stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
      words = [word for word in words if word not  in string.punctuation]
      sentences[i] = ' '.join(words)
      para='.'.join(sentences)
  return para

Perform Lemmatization

In [ ]:
def lemma(para):
  tkn_para=para
  # convert Paragraph to list of sentences
  sentences = nltk.sent_tokenize(para)
  lemmatizer = WordNetLemmatizer()
  # Convert Sentences to words

# Stemming
  for i in range(len(sentences)):
      words = nltk.word_tokenize(sentences[i])
      words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
      words = [word for word in words if word not  in string.punctuation]
      sentences[i] = ' '.join(words)
      para='.'.join(sentences)
  return para

In [23]:
df_converetd=convert_lower(df,'Text')

In [31]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_numerics)

In [30]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_stopwords)

In [29]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_emojis)

In [28]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_punctuation)

In [26]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_urls)

In [27]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_html)

In [32]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(remove_emoticons)

In [38]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(stmng)

In [ ]:
df_converetd['onverted_text']=df_converetd['onverted_text'].apply(lemma)

In [39]:
df_converetd.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,onverted_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought sever vital dog food product found good...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arriv label jumbo salt peanutsth peanu...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confect around centuri light pillowi citru gel...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,look secret ingredi robitussin believ found go...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffi great price wide assort yummi taff...
